In [ ]:
# Sentence Segmentation
## this script records the experiments of en & zh sentence segmentation


In [1]:
import nltk
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import OneClassSVM
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import codecs
import pickle
import time
import nltk.metrics 
from collections import defaultdict
from nltk.classify.util import apply_features, accuracy as eval_accuracy
from nltk.metrics import (BigramAssocMeasures, precision as eval_precision,recall as eval_recall, f_measure as eval_f_measure)

Read training corpus and extract features for English sentence

In [9]:
#training without tokenization 
t0 = time.time()
en = "/home/work/ssCat/sss.en"
with open(en) as inputen:
    tokens = []  #store all the tokes
    boundaries = set()   #store boundary index
    boundaries_token = set()  #store tokens which is a sentence boundary
    offset = 0
    for sent in inputen:
        #nltk tokenization
        #sent = nltk.word_tokenize(sent.decode("utf-8"))
        #print sent
        sent = sent.strip().split(" ")
        #print sent
        if len(sent) >=1:
            boundaries_token.add(sent[-1])
            tokens.extend(sent)
            offset += len(sent)
            boundaries.add(offset-1)
            


def punct_features(tokens, i):
    """The following features are used for English:
    boundary token
    last letter of the boundary token
    last two letters of the boundary token
    previous word of boundary token
    next word of boundary token
    boolean if previous word is digit
    boolean if the first letter of the boundary token is capital
    boolean if the first letter of the next word is capital
    boolean if the first letter of previous word is capital
    boolean if the first letter of the previous 2 word is capital"""
    #print tokens[i]
    return {'punct': tokens[i],\
            'punctLastLetter':tokens[i][-1],\
            'punctL2L':tokens[i][-2:],\
            'prepunct':tokens[i-1],\
            'nextpunct':tokens[i+1],\
            'prepunctdigit':tokens[i-1].isdigit(),\
            'punctCapi': tokens[i][0].isupper(),\
            'next1letterCapi': tokens[i+1][0].isupper(),\
            'pre1LetterCap': tokens[i-1][0].isupper(),\
            'pre2LetterCap': tokens[i-2][0].isupper()}

#sweeping from begine to end, starting from the third word
featuresets = [(punct_features(tokens, i), (i in boundaries))\
               for i in xrange(2, len(tokens)-2) if tokens[i] in boundaries_token]
#split feature data into training developing data and testing data
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
size_dev = int(len(train_set)*0.2)
train_set,dev_set = train_set[size_dev:], train_set[:size_dev]
print "processing data time: ",time.time()-t0,"s"

processing data time:  3.45624399185


In [26]:
#vectorize letter feature into i-k 
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk import compat
def vectorize(labeled_featuresets):
    x, y = list(compat.izip(*labeled_featuresets))
    v = DictVectorizer()
    x =v.fit_transform(x)
    y = LabelEncoder().fit_transform(y)
    return x,y
x,y = vectorize(train_set)
print x.shape

(33755, 23419)


In [10]:
#two layers neural network
import tensorflow as tf
import numpy as np

N, D, H, C = 64, 10, 5, 2

x = tf.placeholder(tf,float32, shape = [None, D])
y = tf.placeholder(tf,float32, shape = [None, C])

w1 = tf.Variable(1e-3 * np.random.randn(D,H),astype(np.float32))
w2 = tf.Variable(1e-3 * np.random.randn(H,C),astype(np.float32))

a = tf.matmul(x,w1)
a_relu = tf.nn.relu(a)
scores = tf.matmul(a_relu,w2)
probs = tf.nn.softmax(scores)
loss = -tf.reduce_sum(y*tf.log(probs))

#for tensorboard
loss_summary = tf.scalar_summary("loss",loss)
w1_hist = tf.histogram_summary('w1',w1)
w2_hist = tf.histogram_summary('w2',w2)

learning_rate = 1e-2
train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)


In [ ]:
xx = np.random.randn(N,D).astype(np.float32)
yy = np.zeros((N,C)).astype(np.float32)
yy[np.arange(N),np.random.randint((C,size=N))] = 1

In [ ]:
with tf.Session() as sess:
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter('/home/work/sstmp/ss_log',sess.graph_def)
    sess.run(tf.initialize_all_tables())
    
    for t in xrange(100):
        summary_str,_,loss_value = sess.run([merged,train_step,loss],
                                            feed_dict={x:xx,y:yy})
        write.add_summary(summar_str,t)
        print "loss value".loss_value

Reading Chinese sentence and extract features

In [11]:
#training with sentence segmentation
t0 = time.time()
en = "/home/work/ssCat/sss.zh"
with open(en) as inputen:
    tokens = []
    boundaries = set()
    boundaries_token = set()
    offset = 0
    for sent in inputen:
        sent = sent.strip().split("") #split into characters
        if len(sent) >=1:
            boundaries_token.add(sent[-1])
            tokens.extend(sent)
            offset += len(sent)
            boundaries.add(offset-1)
#print len(tokens)     
def punct_features(tokens, i):
    """Chinese feature:
    previous two character
    previous word
    boolean previous word is digit
    boundary word
    boolean if boundary word is digit
    boolean if next word is digit
    """
    return {'prev-2word': tokens[i-2],\
            'prev-word': tokens[i-1],\
            'predigit': tokens[i-1].isdigit(),\
            'punct': tokens[i],\
            'punctdigit': tokens[i].isdigit(),\
            'next-word-digit':tokens[i+1].isdigit()}

featuresets = [(punct_features(tokens, i), (i in boundaries))\
               for i in range(1, len(tokens)-1) if tokens[i] in boundaries_token]

size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
size_dev = int(len(train_set)*0.2)
train_set,dev_set = train_set[size_dev:], train_set[:size_dev]
print "processing data time: ",time.time()-t0  

634978
processing data time:  12.5063760281


In [4]:
def evaluate(classifier, test_set, accuracy=True, f_measure=True, precision=True, recall=True):
        
        print("Evaluating {} results...".format(type(classifier).__name__))
        metrics_results = {}
        if accuracy == True:
            accuracy_score = nltk.classify.accuracy(classifier, test_set)
            metrics_results['Accuracy'] = accuracy_score

        gold_results = defaultdict(set)
        test_results = defaultdict(set)
        labels = set()
        for i, (feats, label) in enumerate(test_set):
            labels.add(label)
            gold_results[label].add(i)
            observed = classifier.classify(feats)
            test_results[observed].add(i)

        for label in labels:
            if precision == True:
                precision_score = eval_precision(gold_results[label],
                    test_results[label])
                metrics_results['Precision [{}]'.format(label)] = precision_score
            if recall == True:
                recall_score = eval_recall(gold_results[label],
                    test_results[label])
                metrics_results['Recall [{}]'.format(label)] = recall_score
            if f_measure == True:
                f_measure_score = eval_f_measure(gold_results[label],
                    test_results[label])
                metrics_results['F-measure [{}]'.format(label)] = f_measure_score

        # Print evaluation results (in alphabetical order)
        for result in sorted(metrics_results):
            print('{}: {}'.format(result, metrics_results[result]))

In [12]:
#using libsvm
t1 = time.time()
classifier = SklearnClassifier(LinearSVC()).train(train_set)
print "training time: ",time.time()-t1 
print nltk.classify.accuracy(classifier, dev_set)
print nltk.classify.accuracy(classifier, test_set)
evaluate(classifier, test_set)

training time:  32.6861569881
0.952293641459
0.955475293458
Evaluating SklearnClassifier results...
Accuracy: 0.955475293458
F-measure [False]: 0.975462438656
F-measure [True]: 0.759905977166
Precision [False]: 0.96673695667
Precision [True]: 0.833517495396
Recall [False]: 0.984346862446
Recall [True]: 0.698241283554


In [6]:
t3 = time.time()
classifier = SklearnClassifier(OneClassSVM()).train(train_set)
print "training time: ",time.time()-t3
print nltk.classify.accuracy(classifier, dev_set)
print nltk.classify.accuracy(classifier, test_set)
evaluate(classifier, test_set)

 training time:  4158.4049499
0.119447145724
0.127076350093
Evaluating SklearnClassifier results...
Accuracy: 0.127076350093
F-measure [False]: None
F-measure [True]: 0.225497323376
Precision [False]: None
Precision [True]: 0.127076350093
Recall [False]: 0.0
Recall [True]: 1.0


In [ ]:
t4 = time.time()
classifier = SklearnClassifier(SVC()).train(train_set)
print "training time: ",time.time()-t4 
print nltk.classify.accuracy(classifier, dev_set)
print nltk.classify.accuracy(classifier, test_set)
evaluate(classifier, test_set)

In [33]:
#using naive bayes
t2 = time.time()
classifier = classifier = nltk.NaiveBayesClassifier.train(train_set)
print "training time: ",time.time()-t2
print nltk.classify.accuracy(classifier, dev_set)
print nltk.classify.accuracy(classifier, test_set)
print classifier.show_most_informative_features()
evaluate(classifier, test_set)

training time:  1.87185192108
0.967328177585
0.969388849178
Most Informative Features
                   punct = '?'              True : False  =    118.0 : 1.0
                   punct = ','             False : True   =     87.6 : 1.0
                   punct = 'f'             False : True   =     86.5 : 1.0
                   punct = '.'              True : False  =     69.8 : 1.0
                   punct = '}'              True : False  =     48.5 : 1.0
                   punct = 'o'             False : True   =     46.7 : 1.0
                   punct = '!'              True : False  =     45.2 : 1.0
                   punct = 'u'             False : True   =     39.1 : 1.0
                   punct = '\xa6'           True : False  =     38.0 : 1.0
                   punct = '-'             False : True   =     31.0 : 1.0
None
Evaluating NaiveBayesClassifier results...
Accuracy: 0.969388849178
F-measure [False]: 0.983870815875
F-measure [True]: 0.700262467192
Precision [False]: 0.976

In [ ]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in enumerate(words):
        if word in '.?!;' and i != len(words)-1 and classifier.classify(punct_features(words, i)) == True:             #
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

teststring = "If you have your own collection of text files that you would\
 like to access using the above methods, you can easily load them with\
  the help of NLTK's PlaintextCorpusReader. Check the location of your \
  files on your file system; in the following example, we have taken \
  this to be the directory /usr/share/dict. Whatever the location, \
  set this to be the value of corpus_root."

wl = nltk.word_tokenize(teststring)
ss = segment_sentences(wl)
print len(ss)

* *select even number sentence with sed*

`sed -n 2~2p rall.en > first`
`sed -n 'n;p'`

* *select odd number sentence with sed*

`sed -n 1~2p first >second`
`sed -n 'p;n'`

* *select tandomly about 1% of the total corpus*

`perl -ne 'print if (rand() < .01)' forth > fifth`

* *select sentence with length shorten than 60 tokens*

`awk 'length($0)<60' fifth >sixth`

* *select sentence exceed 80*

`grep '^.\{80\}' file`

`perl -nle 'print if length$_>79' file`

`sed -n '/.\{80\}/p' file`       

In [3]:
#! /bin/python
#-*- coding:utf-8 -*-

import re
import sys
import codecs

def readText(text):
    sentences = []
    for line in text:
	line = line.strip()
	line = sub(line)
	if line != '':
	    sentences.append(line)
    return sentences
	
def sub(text):
    rep = {"/n+": "", "/s+": "","/t+":""} # define desired replacements here
    # use these three lines to do the replacement
    rep = dict((re.escape(k), v) for k, v in rep.iteritems())
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    return text
	
def splitText(senlist):
    for i in senlist:
        split = re.split('("。”")|("？”")|("！”")|("？！")|("！？")|("！！！")|("……")|("？？？")|。|,|("？")|("！")|(";")',i)
        print i
        split = i.split("。")
        print "".join(split)
#f = codecs.open("/home/bear/experiments/30429.txt")
#splitText(readText(f))
tmpl = [u"一双灵巧的手飞舞着操纵着键盘和鼠标，富有节奏的敲击声仿佛是一首轻快的乐章。屏幕上漫天的光华闪过，对手飞扬着血花倒了下去。",u'“呵呵。”叶秋笑了笑，抬手取下了衔在嘴角的烟头。银白的烟灰已经结成了长长一串，但在叶秋挥舞着鼠标敲打着键盘施展操作的过程中却没有被震落分毫。摘下的烟头很快被掐灭在了桌上的一个形状古怪的烟']
splitText(tmpl)

			

一双灵巧的手飞舞着操纵着键盘和鼠标，富有节奏的敲击声仿佛是一首轻快的乐章。屏幕上漫天的光华闪过，对手飞扬着血花倒了下去。


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe3 in position 0: ordinal not in range(128)